# Heirarchical Agent Structure Example

The following code suppresses warning messages to ensure the output is clean and focused on the essential results.

In [ ]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

Imports essential modules from CrewAI and initializes two tools, one for web scraping and one for search engine interaction.Also, sets up environment variables to securely store API keys required for accessing external services.

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Create a search tool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


import os
os.environ["OPENAI_API_KEY"] = 'ADD YOUR API KEY HERE'
os.environ["GROQ_API_KEY"] = 'ADD YOUR API KEY HERE'
os.environ["SERPER_API_KEY"] = 'ADD YOUR API KEY HERE'

Initializes a large language model (LLM) using the specified model and API key for advanced text generation and processing.

In [ ]:
from langchain_groq import ChatGroq

model = 'llama3-groq-70b-8192-tool-use-preview'
llm = ChatGroq(
            temperature=0,
            groq_api_key = os.getenv('GROQ_API_KEY'),
            model_name=model
        )

Defines three agents with specific roles and goals: a Research Analyst, Report Writer, and Report Editor, each leveraging LLM and specialized tools to accomplish their tasks.

In [ ]:
research_analyst_agent = Agent(
    role="Research Analyst",
    goal="Create and analyze research points to provide comprehensive insights on various topics.",
    backstory="Specializing in research analysis, this agent employs advanced methodologies to generate detailed research points and insights. With a deep understanding of research frameworks and a talent for synthesizing information, the Research Analyst Agent is instrumental in delivering thorough and actionable research outcomes.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

report_writer_agent = Agent(
    role="Report Writer",
    goal="Compile the analyzed data into a comprehensive and well-structured research report.",
    backstory="You are skilled at transforming complex information into clear, concise, and informative reports.",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

report_editor_agent = Agent(
    role="Report Editor",
    goal="Review and refine research reports to ensure clarity, accuracy, and adherence to standards.",
    backstory="With a keen eye for detail and a strong background in report editing, this agent ensures that research reports are polished, coherent, and meet high-quality standards. Skilled in revising content for clarity and consistency, the Report Editor Agent plays a critical role in finalizing research outputs.",
    verbose=True,
    llm=llm
)


Defines tasks for data collection, analysis, report writing, and assessment, each assigned to the appropriate agent with clear descriptions and expected outputs.

In [ ]:
# Define tasks
data_collection_task = Task(
    description=(
        "Collect data from relevant sources about the given {topic}."
        "Focus on identifying key trends, benefits, and challenges."
    ),
    expected_output=(
        "A comprehensive dataset that includes recent studies, statistics, and expert opinions."
    ),
    agent=research_analyst_agent,
)

data_analysis_task = Task(
    description=(
        "Analyze the collected data to identify key trends, benefits, and challenges for the {topic}."
    ),
    expected_output=(
        "A detailed analysis report highlighting the most significant findings."
    ),
    agent=research_analyst_agent,
)

report_writing_task = Task(
    description=(
        "Write a comprehensive research report that clearly presents the findings from the data analysis report"
    ),
    expected_output=(
        "A well-structured research report that provides insights about the topic."
    ),
    agent=report_writer_agent,
)

report_assessment_task = Task(
    description=(
        "Review and rewrite the research report to ensure clarity, accuracy, and adherence to standards."
    ),
    expected_output=(
        "A polished, coherent research report that meets high-quality standards and effectively communicates the findings."
    ),
    agent=report_editor_agent,
)


Creates a hierarchical crew with a management LLM and assigns agents to tasks, then kicks off the project.

In [ ]:
# Define the hierarchical crew with a management LLM
research_crew = Crew(
    agents=[research_analyst_agent, report_writer_agent, report_editor_agent],
    tasks=[data_collection_task, data_analysis_task, report_writing_task, report_assessment_task],
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

# Define the input for the research topic
research_inputs = {
    'topic': 'The impact of AI on modern healthcare systems'
}

# Kickoff the project with the specified topic
result = research_crew.kickoff(inputs=research_inputs)


 [2024-08-18 14:24:57][DEBUG]: == Working Agent: Crew Manager
 [2024-08-18 14:24:57][INFO]: == Starting Task: Collect data from relevant sources about the impact of AI on healthcare. Focus on identifying key trends, benefits, and challenges.


> Entering new CrewAgentExecutor chain...
Action: Ask question to coworker
Action Input: {"question": "What are the key trends in AI impacting healthcare?", "context": "Focus on recent studies and statistics.", "coworker": "Research Analyst"}

> Entering new CrewAgentExecutor chain...
Action: Search the internet
Action Input: {"search_query": "recent studies on AI in healthcare"} 


Search results: Title: The potential for artificial intelligence in healthcare - PMC - NCBI
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6616181/
Snippet: There are already a number of research studies suggesting that AI can perform as well as or better than humans at key healthcare tasks, such as diagnosing ...
---
Title: Revolutionizing healthcare: the role of

Converts the crew’s output into a Markdown string for better readability and displays the results in a formatted manner.

In [ ]:
from IPython.display import Markdown

# Convert the CrewOutput object to a Markdown string
result_markdown = result.raw

# Display the result as Markdown
Markdown(result_markdown)

Title: Transforming Healthcare with Artificial Intelligence: Trends, Benefits, and Challenges

Executive Summary:

The integration of artificial intelligence (AI) in healthcare is rapidly evolving, with significant potential to revolutionize patient care, diagnostics, and healthcare management. This report presents a comprehensive analysis of the trends, benefits, and challenges associated with the use of AI in healthcare, drawing on data from reputable sources such as the National Institutes of Health, IBM, and the American Cancer Society.

Key Trends in AI in Healthcare:

1. AI's Potential to Outperform Humans: Studies have shown that AI can perform as well as or better than humans at key healthcare tasks, such as diagnosing diseases, improving diagnostics and analysis of patient data, detecting diseases more accurately and in their early stages, enhancing the healthcare user experience, and enabling services to be more efficient and predictive of patients' needs.

2. Data Analysis and Diagnostics: AI's ability to analyze large amounts of data quickly and accurately makes it an invaluable tool for improving diagnostics and patient care. AI algorithms can identify patterns and anomalies in patient data, leading to more accurate diagnoses and better treatment outcomes.

3. Patient Care and Monitoring: AI can significantly enhance patient care by providing real-time monitoring, personalized treatment plans, and predictive analytics to anticipate patient needs. This can lead to better patient outcomes and reduced healthcare costs.

4. Healthcare Efficiency and Management: AI can streamline healthcare operations, improve patient flow, and optimize resource allocation, resulting in more efficient and cost-effective healthcare services.

Benefits of AI in Healthcare:

1. Improved Diagnostics and Patient Care: AI's analytical capabilities can lead to more accurate diagnoses, better patient care, and improved treatment outcomes.

2. Cost Reduction: AI can help reduce healthcare costs by improving diagnostics, reducing unnecessary tests and treatments, and enhancing patient care.

3. Real-Time Patient Monitoring: AI's ability to provide real-time monitoring enables healthcare providers to respond promptly to patient needs, improving patient outcomes and reducing healthcare costs.

4. Democratization of Healthcare: AI has the potential to democratize healthcare by making high-quality care accessible to more people, regardless of geographical or socioeconomic factors.

Challenges and Ethical Concerns in AI in Healthcare:

1. Ethical Concerns and Data Privacy Issues: The use of AI in healthcare raises ethical concerns, particularly regarding data privacy and security. Ensuring that patient data is protected and used ethically is crucial.

2. Job Displacement and Human-AI Collaboration: The integration of AI in healthcare may lead to job displacement, highlighting the need for healthcare professionals to develop new skills and collaborate effectively with AI systems.

3. Transparency and Explainability: AI algorithms must be transparent and explainable to ensure trust and confidence in AI-driven decisions.

4. Addressing Disparities in Healthcare: AI has the potential to exacerbate existing disparities in healthcare if not designed and implemented carefully. It is essential to ensure that AI systems are equitable and accessible to all patients.

Conclusion:

The use of AI in healthcare offers significant potential benefits, including improved diagnostics, better patient care, cost reduction, and democratization of healthcare. However, it also presents challenges and ethical concerns that must be addressed to ensure the successful integration of AI in healthcare. By understanding these trends, benefits, and challenges, healthcare providers and policymakers can work towards creating a more efficient, equitable, and effective healthcare system that leverages the power of AI while protecting patient rights and promoting better health outcomes.